# Generates stats of the various categories 

In [ ]:
import pandas as pd

In [ ]:
def detector(line):
    if line.startswith("<malay>") and line.endswith("</malay>"):
        if line.count("<malay>") == 1:
            return "MS"
        else:
            return "CS"
    else:
        if line.count("<malay>") >= 1:
            return "CS"
        else:
            return "EN"

In [ ]:
def word_analyzer(word, tag):
    ar = ""
    tag2= tag
    if tag == "<malay>":
        tag2 = "</malay>"
    if tag == "(":
        tag2 = ")"
    if tag == "[":
        tag2 = "]"
    if tag == "<":
        tag2 = ">"
    if tag == "{":
        tag2 = "}"
    n = len(tag)
    k = len(tag2)
    if word.startswith(tag) and word.endswith(tag2):
            ar = word[n:-k]
    elif word.startswith(tag):
            ar = word[n:]
    elif word.endswith(tag2):
            ar = word[:-k]
    return ar

In [ ]:
def data_cleanser(string):
    words = string.split()
    eng_ar = []
    mal_ar = []
    flag2 = False
    eng = ""
    mal = ""
    ar = []
    sent = ""
    for i in range(len(words)):
        word = words[i]
        if "_" in word:
            wrd = ""
            for j in range(len(word)):
                if word[j] != "_":
                    wrd += word[j]
            word = wrd
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("<malay>") or word.endswith("</malay>"):
            word2 = word
            if word.startswith("<malay>"):
                flag2 = True
                eng_ar.append(eng)
                eng = ""
            word = word_analyzer(word, "<malay>")
            mal += word+ " "
            if word2.endswith("</malay>"):
                flag2 = False
                mal_ar.append(mal)
                mal = ""
        elif word.startswith("(") or word.endswith(")"):
            word = word_analyzer(word, "(")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("[") or word.endswith("]"):
            word = word_analyzer(word, "[")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("<") or word.endswith(">"):
            word = word_analyzer(word, "<")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("{") or word.endswith("}"):
            word = word_analyzer(word, "{")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("!") or word.endswith("!"):
            word = word_analyzer(word, "!")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("#") or word.endswith("#"):
            word = word_analyzer(word, "#")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.endswith("~"):
            continue
        else:
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        ar.append(word)
    if mal == "" and eng != "":
        eng_ar.append(eng)
    elif mal != "" and eng == "":
        mal_ar.append(mal)
    return eng_ar, mal_ar, " ".join(ar)

In [ ]:
def get_length(eng_ar, mal_ar, mal_wrd, eng_wrd):
    for eng in eng_ar:
        eng_wrd += len(eng.split())
    for mal in mal_ar:
        mal_wrd += len(mal.split())
    return mal_wrd, eng_wrd

In [ ]:
def generate_stats(folder_name, name):
    with open(folder_name+"/"+name+".txt") as f1:
        text = f1.readlines()
    f1.close()
    cs_n = 0
    mal_n = 0
    eng_n = 0
    mal_wrd = 0
    eng_wrd = 0
    cs = ""
    mal = ""
    eng = ""
    for i in range(len(text)):
        string = text[i][:-1]
        lang = detector(string)
        eng_ar, mal_ar, sent = data_cleanser(string)
        mal_wrd, eng_wrd = get_length(eng_ar, mal_ar, mal_wrd, eng_wrd)
        if lang == "CS":
            cs += sent + "\n"
            cs_n += 1
        if lang == "MS":
            mal += sent + "\n"
            mal_n += 1
        if lang == "EN":
            eng += sent + "\n"
            eng_n += 1
    f1 = open(folder_name+"/cs_"+name+".txt",'w+')
    f1.write(cs)
    f1.close()
    f1 = open(folder_name+"/eng_"+name+".txt",'w+')
    f1.write(eng)
    f1.close()
    f1 = open(folder_name+"/mal_"+name+".txt",'w+')
    f1.write(mal)
    f1.close()

    return cs_n, mal_n, eng_n, mal_wrd, eng_wrd

In [ ]:
column = ["File", "# CS Sentences", "# English Sentences", "# Malay Sentences", "# Total Sentences", "% CS Sentences", "# English Words", "# Malay Words", "# Total Words", "Substitution Rate(English)", "CMI"]
df = pd.DataFrame(columns = column)

In [ ]:
df.loc[len(df.index)] = ["Based on Age","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Age", "complete_18_30")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["18 to 30 years", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Age", "complete_31_45")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["31 to 45 years", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Age", "complete_46_more")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["46 and more ", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]


In [ ]:
df.loc[len(df.index)] = ["Based on Gender","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Gender", "complete_Male")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Male", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Gender", "complete_Female")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Female", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]


In [ ]:
df.loc[len(df.index)] = ["Based on Relationship","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Relationship", "complete_Family")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Family", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Relationship", "complete_Friends")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Friends", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Relationship", "complete_Strangers")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Strangers", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]


In [ ]:
df.loc[len(df.index)] = ["Based on Ethnicity","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Ethnicity", "complete_Malay")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Malay", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Ethnicity", "complete_Indian")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Indian", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Ethnicity", "complete_Chinese")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Chinese", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

In [ ]:
df.loc[len(df.index)] = ["Based on Education","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Education", "complete_School")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["School", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Education", "complete_Polytechnic")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Polytechnic", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Education", "complete_University")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["University", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]


In [ ]:
df.loc[len(df.index)] = ["Based on Employment","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Employment", "complete_Students")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Students", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Employment", "complete_Unemployed")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Unemployed", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Employment", "complete_Employed")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
df.loc[len(df.index)] = ["Employed", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]


In [ ]:
df.loc[len(df.index)] = ["Based on Dominant Language","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Dominant_Language", "complete_Malay")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
if tot_n != 0:
    df.loc[len(df.index)] = ["Malay", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Dominant_Language", "complete_English")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
if tot_n != 0:
    df.loc[len(df.index)] = ["English", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("Dominant_Language", "complete_Other")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
if tot_n != 0:
    df.loc[len(df.index)] = ["Other", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

In [ ]:
df.loc[len(df.index)] = ["Based on First Language","","","","","","","","","",""]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("First_Language", "complete_Malay")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
if tot_n != 0:
    df.loc[len(df.index)] = ["Malay", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("First_Language", "complete_English")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
if tot_n != 0:
    df.loc[len(df.index)] = ["English", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]

cs_n, mal_n, eng_n, mal_wrd, eng_wrd = generate_stats("First_Language", "complete_Other")
tot_n = cs_n + mal_n + eng_n
tot_wrd = eng_wrd + mal_wrd
if tot_n != 0:
    df.loc[len(df.index)] = ["Other", cs_n, eng_n, mal_n, tot_n, round(cs_n*100/tot_n, 2), eng_wrd, mal_wrd, tot_wrd, round(eng_wrd*100/tot_wrd, 2), round(100 * (1 - (max(eng_wrd/tot_wrd, mal_wrd/tot_wrd)) ), 2) ]


In [ ]:
df.to_excel("Results.xlsx", index = False)